# **Ressources**

## Software links
- **Github** : https://github.com/brainets/frites/
- **Documentation** : https://brainets.github.io/frites/

## Papers

### Information theory
- Timme and Lapish, 2018, [A tutorial for information theory in neuroscience](https://www.eneuro.org/content/eneuro/early/2018/06/29/ENEURO.0052-18.2018.full.pdf)
- RAA Ince et al., 2017, [A statistical framework for neuroimaging data analysis based on mutual information estimated via a gaussian copula: Gaussian Copula Mutual Information](https://onlinelibrary.wiley.com/doi/pdfdirect/10.1002/hbm.23471)

### Statistics
- Penny and holmes, 2007, [Random effects analysis](https://citeseerx.ist.psu.edu/viewdoc/download?doi=10.1.1.380.3435&rep=rep1&type=pdf)
- Nichols and Holmes, 2002, [Nonparametric permutation tests for functional neuroimaging: a primer with examples](https://scholar.google.com/scholar_url?url=https://onlinelibrary.wiley.com/doi/pdfdirect/10.1002/hbm.1058&hl=fr&sa=T&oi=gsb-gga&ct=res&cd=0&d=7639226016862188227&ei=kuBzYo-jKJuVy9YPlcKL6Ac&scisig=AAGBfm23SI_p43GWskhvI6-GUZ432i-HTg)
- Fries and Maris, 2021 (Biorxiv), [What to do if N is two?](https://arxiv.org/pdf/2106.14562)
- Combrisson et al., 2021 (Biorxiv), [Group-level inference of information-based measures for the analyses of cognitive brain networks from neurophysiological data](https://www.biorxiv.org/content/10.1101/2021.08.14.456339.full.pdf)

### Functional Connectivity

- Bastos and Schoffelen, 2015, [A Tutorial Review of Functional Connectivity Analysis Methods and Their Interpretational Pitfalls](https://www.researchgate.net/profile/Jan-Mathijs-Schoffelen/publication/290985451_A_Tutorial_Review_of_Functional_Connectivity_Analysis_Methods_and_Their_Interpretational_Pitfalls/links/569f8dec08aee4d26ad267db/A-Tutorial-Review-of-Functional-Connectivity-Analysis-Methods-and-Their-Interpretational-Pitfalls.pdf)

## YouTube channel (why not)

[Mike X Cohen youTube channel](https://www.youtube.com/channel/UCUR_LsXk7IYyueSnXcNextQ) : data analysis, time- phase-locked, evoked vs. induced, time-frequency decomposition, statistics etc.


---
# **Import libraries**

In [ ]:
import numpy as np
import xarray as xr
import pandas as pd
import os

from frites import set_mpl_style

import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
import seaborn as sns

set_mpl_style()

###############################################################################
repl_outc = {-2: '-1€', -1: '-0€', 1: '+0€', 2: '+1€'}
root = os.path.join('..', 'dataset')
###############################################################################

---
# **Data manipulation**

## Load and plot the data coming from a single subject

In [ ]:
###############################################################################
subject_nb = 2
###############################################################################

# load the high-gamma activity
file_hga = os.path.join(root, 'hga', f'hga_s-{subject_nb}.nc')
hga = xr.load_dataarray(file_hga)
hga.name = 'HGA (subject_2)'
hga

## Data selection (temporal and spatial)

In [ ]:
# single time point selection
hga.sel(times=0.)

# temporal interval selection
hga.sel(times=slice(0., 1.))

# channel selection
hga.sel(channels='O3-O2')

# combining channel and temporal selection
hga.sel(channels='O3-O2', times=slice(0., 1.))

## Operation on DataArray

### `mean` operation

In [ ]:
# mean over trials
hga.mean('trials')

# mean over channels
hga.mean('channels')

# mean over channels and trials
hga.mean(['trials', 'channels'])


### `groupby` operation 

In [ ]:
hga.groupby('trials').mean()

---
# **Data plotting**

## Plot the overal mean (trials and channels)

In [ ]:
hga.mean(['trials', 'channels']).plot()
plt.axvline(0, color='C3')
plt.title('Mean HGA over trials and channels');

## Plot the mean over trials but **per** channel

In [ ]:
fg = hga.mean('trials').plot(x='times', col='channels', col_wrap=6)
_ = [ax.axvline(0, color='C3') for ax in np.ravel(fg.axes)]

## Plot the mean HGA across contacts, per channel and per outcome

In [ ]:
fg = hga.groupby('trials').mean().plot(x='times', col='channels', hue='trials', col_wrap=6)
_ = [ax.axvline(0, color='C3') for ax in np.ravel(fg.axes)]

---
# **Group-level statistics**

## Prerequisites

### Loading functions
Functions for loading the data of a single subject or multiple subjects + attach parcel name in place of channel names

In [ ]:
from mne.utils import ProgressBar

###############################################################################
###############################################################################
#                 Load the data of a single subject
###############################################################################
###############################################################################

def load_ss(subject_nb, use_roi=False):
    """Load the data of a single subject.
    
    Parameters
    ----------
    subject_nb : int
        Subject number [0, 12]
    
    Returns
    -------
    hga : xarray.DataArray
        Xarray containing the high-gamma activity
    anat : pandas.DataFrame
        Table containing the anatomical informations
    beh : pandas.DataFrame
        Table containing the behavioral informations
    """
    # load the high-gamma activity
    file_hga = os.path.join(root, 'hga', f'hga_s-{subject_nb}.nc')
    hga = xr.load_dataarray(file_hga)

    # load the name of the brain regions
    file_anat = os.path.join(root, 'anat', f'anat_s-{subject_nb}.xlsx')
    anat = pd.read_excel(file_anat)

    # load the behavior
    file_beh = os.path.join(root, 'beh', f'beh_s-{subject_nb}.xlsx')
    beh = pd.read_excel(file_beh)
    
    if not use_roi:
        return hga, anat, beh
    if use_roi:
          hga = hga.rename(channels='roi')
          hga['roi'] = list(anat['roi'])
          return hga


###############################################################################
###############################################################################
#                 Load the data of multiple subjects
###############################################################################
###############################################################################

def load_ms(s_range=[0, 11], model='outcome', condition='rew',
            space='roi', mean_roi=True, prepend_suj_to_ch=True):
    """Load multiple subjects.
    
    Parameters
    ----------
    s_range : int or list
        Subjects to load. Use either an integer (e.g. 7) to load a single
        subject or a range of subjects (e.g. [5, 10])
    model : {'outcome', 'pe', 'rt'}
        Model to use. Use either :
        
            * 'outcome' : find differences in the neural activity between the
              outcomes
            * 'pe' : find regions with an activity correlating with the
              prediction error
    condition : {'rew', 'pun', 'context', 'null'}
        Condition to load. Use either :
        
            * 'rew' : for outcomes {+0€; +1€}
            * 'pun' : for outcomes {-1€; -0€}
            * 'context' : for outcomes {-1€; +1€}
            * 'null' : for outcomes {-0€; +0€}
    space : {'channels', 'roi'}
        Specify if the spatial dimension should be described with channel names
        or with brain region names
    mean_roi : bool
        Specify if you want to take the mean high-gamma activity inside a brain
        region
    prepend_suj_to_ch : bool
        Add subject name to each channel name
    
    Returns
    -------
    hga : list
        List of high-gamma activity across subjects
    """
    # inputs checking
    if isinstance(s_range, int):
        s_range = [s_range, s_range]
    s_range[1] += 1
    s_range[0], s_range[1] = max(s_range[0], 0), min(s_range[1], 12)
    mesg = f"Subject %i | model={model} | condition={condition} | space={space}"
    pbar = ProgressBar(range(s_range[0], s_range[1]), mesg=mesg % 0)
    model = model.lower()
    assert space in ['channels', 'parcels', 'roi']
    
    # get the code of the condition
    outc = {
        'rew': (+1, +2),
        'pun': (-2, -1),
        'context': (-2, +2),
        'null': (-1, +1)
    }[condition]
    
    # get the behavioral column to use
    col = {
        'outcome': 'code',
        'pe': 'PE',
        'rt': 'RT'
    }[model]
    
    # load the data
    hga = []
    for n_s in range(s_range[0], s_range[1]):
        pbar._tqdm.desc = mesg % n_s
        # load the data of a single subject
        _hga, _anat, _beh = load_ss(n_s)
        _outc = _hga['trials'].data
        _ch = _hga['channels'].data
        
        # replace trial dimension with the model
        _hga = _hga.rename(trials=model)
        _hga[model] = list(_beh[col])
        
        # get which outcome to keep
        keep_outc = np.logical_or(_outc == outc[0], _outc == outc[1])
        _hga = _hga[keep_outc, ...]
        
        # replace with brain regions
        if space in ['parcels', 'roi']:
            _hga = _hga.rename(channels=space)
            _hga[space] = list(_anat['roi'])
            
            # take the mean of the hga per parcel
            if mean_roi:
                _hga = _hga.groupby(space).mean(space)
        elif prepend_suj_to_ch and (space == 'channels'):
            # prepend subject number to channel name
            _hga['channels'] = [f"suj{n_s}/{c}" for c in _ch]
        
        # ascontinuous array
        _hga.data = np.ascontiguousarray(_hga.data)
        
        hga.append(_hga)
        pbar.update_with_increment_value(1)

    return hga


###############################################################################
###############################################################################
#                      Plot results of the stats
###############################################################################
###############################################################################

def plot_mi(mi, pv, color='C0', p=0.05):
    """Plot the results of the statistics.
    """
    # plot effect size
    fg = mi.plot(x='times', col='roi', color=color)
    fig = plt.gcf()
    _ = [ax.axvline(0., color='C3') for ax in np.ravel(fg.axes)]

    # plot the results of the stats
    cl = xr.full_like(pv, 1.1 * mi.data.max())
    cl.data[pv.data >= p] = np.nan

    # plot the significant clusters
    for n_r, r in enumerate(mi['roi'].data):
        plt.sca(np.ravel(fg.axes)[n_r])
        plt.plot(mi['times'].data, cl.sel(roi=r).data, color='C5', lw=4)
        plt.xlabel('Times (s)')
        if n_r == 0: plt.ylabel('MI (bits)')
    
    # add the legend
    custom_lines = [
        Line2D([0], [0], color="C5", lw=6)
    ]
    titles = [f"p < {p}"]
    plt.legend(
        custom_lines, titles, ncol=1, bbox_to_anchor=(1., 0.7), fontsize=20,
        bbox_transform=fig.transFigure, title=None,
    );  

### Understanding the loading function

#### Select the contrast

In [ ]:
# Differences of HGA according to the outcome values : +1€ vs. +0€ (rewarding condition)
_ = load_ms(model='outcome', condition='rew')

# Differences of HGA according to the outcome values : -1€ vs. -0€ (punishment condition)
_ = load_ms(model='outcome', condition='pun')

# Correlation between the HGA and the PE during the rewarding condition
_ = load_ms(model='pe', condition='rew')

# Correlation between the HGA and the PE during the punishment condition
_ = load_ms(model='pe', condition='pun')

#### Understanding the output

In [ ]:
hga_ms = load_ms(model='outcome', condition='rew')

In [ ]:
# hga_ms

### Plot the data across subjects

In [ ]:
# load and plot the data for the rewarding condition
hga_ms = load_ms(model='outcome', condition='rew')
hga_ms = xr.concat([k.groupby('outcome').mean() for k in hga_ms], 'roi').groupby(
    'roi').mean()
hga_ms['outcome'] = [repl_outc[o] for o in hga_ms['outcome'].data]
hga_ms.name = 'HGA'
fg = hga_ms.plot(x='times', hue='outcome', col='roi')
_ = [ax.axvline(0, color='C3') for ax in np.ravel(fg.axes)]
plt.gcf().suptitle("Mean HGA over the subjects during REW condition",
                   fontweight='bold', fontsize=22, y=1.1);

# load and plot the data for the rewarding condition
hga_ms = load_ms(model='outcome', condition='pun')
hga_ms = xr.concat([k.groupby('outcome').mean() for k in hga_ms], 'roi').groupby(
    'roi').mean()
hga_ms['outcome'] = [repl_outc[o] for o in hga_ms['outcome'].data]
hga_ms.name = 'HGA'
fg = hga_ms.plot(x='times', hue='outcome', col='roi')
_ = [ax.axvline(0, color='C3') for ax in np.ravel(fg.axes)]
plt.gcf().suptitle("Mean HGA over the subjects during PUN condition",
                   fontweight='bold', fontsize=22, y=1.1);

## `DatasetEphy` : multi-subjects container

In [ ]:
from frites.dataset import DatasetEphy

# load the data of all of the subjects during the rewarding condition
hga_ms = load_ms(model='outcome', condition='rew')
ds = DatasetEphy(hga_ms, y='outcome', times='times', roi='roi')
ds


## Contrasting HGA between conditions

### $HGA_{+1€} vs. HGA_{+0€}$

In [ ]:
from frites.workflow import WfMi

# define a workflow of mutual-information
wf = WfMi(mi_type='cd', inference='ffx')

# run the workflow
mi, pv = wf.fit(ds, n_perm=200)

plot_mi(mi, pv, color='C1')

### $HGA_{-1€} vs. HGA_{-0€}$

In [ ]:
# load the data of all of the subjects during the punishing condition
hga_ms = load_ms(model='outcome', condition='pun')
ds = DatasetEphy(hga_ms, y='outcome', times='times', roi='roi')

wf = WfMi(mi_type='cd', inference='ffx')
mi, pv = wf.fit(ds, n_perm=200)

plot_mi(mi, pv, color='C0')

### Relation between the HGA and the Reward Prediction Error (RPE)

In [ ]:
# load the data of all of the subjects during the rewarding condition
hga_ms = load_ms(model='pe', condition='rew')
ds = DatasetEphy(hga_ms, y='pe', times='times', roi='roi')

# this tim we are using the continuous-continuous MI
wf = WfMi(mi_type='cc', inference='ffx')
mi, pv = wf.fit(ds, n_perm=200, n_jobs=-1, random_state=0)

plot_mi(mi, pv, color='C1')
plt.ylim(-0.001, 0.05);

### Relation between the HGA and the Punishment Prediction Error (PPE)

In [ ]:
# load the data of all of the subjects during the rewarding condition
hga_ms = load_ms(model='pe', condition='pun')
ds = DatasetEphy(hga_ms, y='pe', times='times', roi='roi')

# this tim we are using the continuous-continuous MI
wf = WfMi(mi_type='cc', inference='ffx')
mi, pv = wf.fit(ds, n_perm=200, n_jobs=-1, random_state=0)

plot_mi(mi, pv, color='C0')
plt.ylim(-0.001, 0.05)

**Conclusion**


1.   **aINS** and **dlPFC** : more Punishment related
2.   **lOFC** and **vmPFC** : more Reward related

---
# **Functional connectivity (FC)**

## Across-time points static FC

In [ ]:
from frites.conn import conn_dfc

# load the data of a single subject
hga = load_ss(2, use_roi=True).sel(times=slice(0., 1.5))

# compute the MI between pairs of channels
dfc = conn_dfc(hga, roi='roi', times='times', roi_relation='both') # roi_relation = {'both', 'intra', 'inter'}

In [ ]:
dfc

## Across-time points dynamic FC

### Define sliding windows

In [ ]:
from frites.conn import define_windows, plot_windows

# load the data of a single subject
hga = load_ss(2, use_roi=True)

# get the time vector
times = hga['times'].data

# define sliding windows
win, _ = define_windows(times, slwin_len=0.4, slwin_step=0.25)

# plot the sliding windows
plot_windows(times, win, title='Sliding windows example')
plt.autoscale(tight=True)

### Compute FC within each window

In [ ]:
dfc = conn_dfc(hga, roi='roi', times='times', roi_relation='intra', win_sample=win)
dfc

## FC across all subjects

### Static FC

In [ ]:
# load the data of all of the subjects for outcomes {-1€, +1€}
hga_ms = load_ms(mean_roi=False, condition='context')

dfc = []
for hga in hga_ms:
  # skip if the subject has a unique roi
  if len(np.unique(hga['roi'])) == 1: continue
  print(hga.name)

  # select the first 500ms
  hga = hga.sel(times=slice(0., .5))

  # compute the dfc for this single subject
  _dfc = conn_dfc(hga, times='times', roi='roi', roi_relation='inter',
                  verbose=False)

  # replace trial names
  _dfc['trials'] = [repl_outc[k] for k in _dfc['trials'].data]

  # mean over trials per condition
  _dfc = _dfc.groupby('trials').mean()

  dfc.append(_dfc.to_dataframe('FC'))

dfc = pd.concat(dfc).reset_index()

In [ ]:
plt.figure(figsize=(12, 8))
sns.boxplot(
    data=dfc, x='roi', y='FC', hue='trials', showfliers=False,
    hue_order=['-1€', '+1€']
);
plt.title("Static FC estimated between [0., 500]ms for -1€ and +1€ outcomes",
          fontweight='bold');

### Dynamic FC

In [ ]:
# load the data of all of the subjects for outcomes {-1€, +1€}
hga_ms = load_ms(mean_roi=False, condition='context')

# define the sliding windows
win, _ = define_windows(times, slwin_len=0.4, slwin_step=0.02)

dfc = []
for hga in hga_ms:
  # skip if the subject has a unique roi
  if len(np.unique(hga['roi'])) == 1: continue
  print(hga.name)

  # compute the dfc for this single subject
  _dfc = conn_dfc(hga, times='times', roi='roi', roi_relation='inter',
                  verbose=False, win_sample=win)

  # replace trial names
  _dfc['trials'] = [repl_outc[k] for k in _dfc['trials'].data]

  # mean over trials and brain regions
  _dfc = _dfc.groupby('trials').mean()

  dfc.append(_dfc)

dfc = xr.concat(dfc, 'roi')

In [ ]:
dfc_gp = dfc.groupby('roi').mean().sel(trials=['-1€', '+1€'])
dfc_gp.name = 'DFC'
fg = dfc_gp.plot(x='times', hue='trials', col='roi', col_wrap=3)
_ = [ax.axvline(0., color='C3') for ax in np.ravel(fg.axes)]
plt.gcf().suptitle("Dynamic FC for -1€ and +1€ outcomes", fontweight='bold',
                   fontsize=22, y=1.06);

## Single-trial coherence

### Simulated data

In [ ]:
from frites.simulations import StimSpecAR

ar = StimSpecAR()
sim = ar.fit(ar_type='osc_40', n_epochs=300, n_std=5, n_stim=1)

### Compute the single-trial coherence

In [ ]:
from frites.conn import conn_spec

freqs = np.linspace(10, 70, 50)

conn = conn_spec(
    sim, roi='roi', times='times', freqs=freqs, sfreq=200., n_cycles=freqs / 4,
    sm_times=.2, mode='multitaper', mt_bandwidth=4
).squeeze().sel(times=slice(-.4, .75))

In [ ]:
plt.figure(figsize=(20, 7))
plt.subplot(121)
conn.mean('trials').plot(x='times', y='freqs', cmap='RdBu_r')
plt.axvline(0., color='w', lw=3)
plt.axhline(40, linestyle='--', color='w')
plt.xlabel('Times (s)'), plt.ylabel('Frequencies (Hz)')
plt.title("Mean over the trials of the coherence", fontweight='bold')

plt.subplot(122)
conn_c = conn.sel(freqs=40., method='nearest')
conn_c['trials'] = np.arange(300)
conn_c.plot(x='times', y='trials', cmap='RdBu_r')
plt.axvline(0., color='w', lw=3)
plt.xlabel('Times (s)'), plt.ylabel('Frequencies (Hz)')
plt.title("Single-trial coherence at 40hz", fontweight='bold');